# Notebook 2: Burgers' Equation

$$
\partial_t u - \nu \partial_x^2 u  = -u \partial_x u
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import dedalus.public as d3
from load_d3 import load_d3
import os 

In [ ]:
nx = 1024
Lx = 10

# Bases
xcoord = d3.Coordinate('x')
dist = d3.Distributor(xcoord, dtype=np.float64)
xbasis = d3.RealFourier(xcoord, nx, bounds=(0, Lx), dealias=1)

# Fields
u = dist.Field(name='u', bases=xbasis)

# Problem
problem = d3.IVP([u,], namespace=locals())

In [ ]:
# Substitutions
dx = lambda A: d3.Differentiate(A, xcoord)

In [ ]:
nu = 8e-4

In [ ]:
# Add main equation, with linear terms on the LHS and nonlinear terms on the RHS
problem.add_equation("dt(u) - nu*dx(dx(u))   = u*dx(u)")

In [ ]:
# Build solver
solver = problem.build_solver(d3.RK222)

In [ ]:
x = dist.local_grid(xbasis)
n = 20
u['g'] = np.log(1 + np.cosh(n)**2/np.cosh(n*(x-0.2*Lx))**2) / (2*n)

In [ ]:
solver.stop_sim_time = 5

In [ ]:
if not os.path.exists('burgers'):
    os.mkdir('burgers')
check = solver.evaluator.add_file_handler('burgers/checkpoints', iter=10, parallel='gather')
check.add_tasks(solver.state)

In [ ]:
# Main loop
timestep = 1e-3
while solver.proceed:
    solver.step(timestep)
    if solver.iteration % 200 == 0:
        print('Completed iteration {}'.format(solver.iteration))

In [ ]:
df = load_d3("burgers/checkpoints/checkpoints_s1.h5")

In [ ]:
a = df['u'].plot(x="x",y="t")

In [ ]:
df['u'].shape

In [ ]:
df['u'][::50].plot(x="x", col="t",col_wrap=3)

In [ ]:
df['u'][-1].plot(x="x")
plt.xlim(8,10)